In [ ]:
from astropy.io import ascii, fits
import os
import requests
import importlib
import io
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from skimage import measure
from numpy.polynomial.legendre import Legendre


In [ ]:
archive_api = 'https://archive-api.lco.global/frames/'

In [ ]:
def download_frame(frame_id, filename, archive_api):
    response = requests.get(archive_api + str(frame_id)).json()
    buffer = io.BytesIO()
    buffer.write(requests.get(response['url'], stream=True).content)
    buffer.seek(0)
    return buffer

Download the flat fields to get the location of the orders

In [ ]:
skyflat_files = ascii.read(os.path.join(importlib.resources.files('banzai_floyds.tests'), 'data/test_skyflat.dat'))
for skyflat in skyflat_files:
    skyflat_info = dict(skyflat)    
    skyflat_hdu = fits.open(download_frame(skyflat_info['frameid'], skyflat_info['filename'], archive_api))

    # Munge the data to be OBSTYPE SKYFLAT
    skyflat_hdu['SCI'].header['OBSTYPE'] = 'SKYFLAT'
    skyflat_name = skyflat_info["filename"].replace("x00.fits", "f00.fits")
    filename = os.path.join('manual_reduction', f'{skyflat_name}')
    skyflat_hdu.writeto(filename, overwrite=True)
    skyflat_hdu.close()

In [ ]:
import plotly.express as px

In [ ]:
ogg_hdu = fits.open('manual_reduction/ogg2m001-en06-20190329-0018-f00.fits.fz')
px.imshow(ogg_hdu['SCI'].data.astype(float), origin='lower')

In [ ]:
convolved = np.sum(sliding_window_view(ogg_hdu['SCI'].data, window_shape = 95, axis=0), axis=2)
fig = px.imshow(convolved, origin='lower')
fig.show()

In [ ]:
# Do a makeshift max filter in only the y-direction
window_view_summed = sliding_window_view(convolved, window_shape=7, axis=0)
max_filtered = np.max(window_view_summed, axis=2) == convolved[3:-3]

# Detect the maxima and only filter out everything that isn't the main order centers
ogg_center_labels = measure.label(max_filtered)
ogg_center_properties = measure.regionprops(ogg_center_labels)
for prop in ogg_center_properties:
    if prop.area > 500:
        edge = ogg_center_labels == prop.label
        fig = px.imshow(edge, origin='lower')
        fig.show()

Fit legendre polynomials to the edges, test to make sure that 99% of the pipeline region pixels fall between those two curves

In [ ]:
orders = {'coj': {1: {}, 2: {}}, 'ogg': {1: {}, 2: {}}}

In [ ]:
x2d, y2d = np.meshgrid(np.arange(ogg_hdu['SCI'].data.shape[1]), np.arange(ogg_hdu['SCI'].data.shape[0]))
order_id = 1
for prop in ogg_center_properties:
    if prop.area > 500:
        edge = np.zeros(ogg_hdu['SCI'].data.shape, dtype=bool)
        # We have to convert back to original image coordinates
        # We do this by taking the 95 pixel wide filter size and the 7 pixel wide max filter size
        # and padding the filtered data by the half width of each
        edge[95//2 + 7//2:-95//2 - 7//2 + 1] = ogg_center_labels == prop.label
        # Fit a Legendre polynomial to the center of the order
        x, y = x2d[edge], y2d[edge]
        best_fit = Legendre.fit(x, y, 5)
        coeffs = ",".join(map(str, best_fit.coef))
        domain = ",".join(map(str, best_fit.domain))
        range_str = ",".join(map(str, best_fit.window))
        print(f'Legendre(coef=[{coeffs}], domain=[{domain}], range=[{range_str}])')   
        orders['ogg'][order_id]['coeffs'] = best_fit.coef
        orders['ogg'][order_id]['domain'] = best_fit.domain
        orders['ogg'][order_id]['window'] = best_fit.window
        order_id += 1


In [ ]:
# Do the same procedure for COJ
coj_hdu = fits.open('manual_reduction/coj2m002-en12-20220313-0002-f00.fits.fz')
px.imshow(coj_hdu['SCI'].data.astype(float), origin='lower')

In [ ]:
convolved = np.sum(sliding_window_view(coj_hdu['SCI'].data, window_shape = 95, axis=0), axis=2)

# Do a makeshift max filter in only the y-direction
window_view_summed = sliding_window_view(convolved, window_shape=7, axis=0)
max_filtered = np.max(window_view_summed, axis=2) == convolved[3:-3]

# Detect the maxima and only filter out everything that isn't the main order centers
coj_center_labels = measure.label(max_filtered)
coj_center_properties = measure.regionprops(coj_center_labels)
for prop in coj_center_properties:
    if prop.area > 500:
        edge = coj_center_labels == prop.label
        fig = px.imshow(edge, origin='lower')
        fig.show()

In [ ]:
x2d, y2d = np.meshgrid(np.arange(coj_hdu['SCI'].data.shape[1]), np.arange(coj_hdu['SCI'].data.shape[0]))
order_id = 1
for prop in coj_center_properties:
    if prop.area > 500:
        edge = np.zeros(coj_hdu['SCI'].data.shape, dtype=bool)
        # We have to convert back to original image coordinates
        # We do this by taking the 95 pixel wide filter size and the 7 pixel wide max filter size
        # and padding the filtered data by the half width of each
        edge[95//2 + 7//2:-95//2 - 7//2 + 1] = coj_center_labels == prop.label
        # Fit a Legendre polynomial to the center of the order
        x, y = x2d[edge], y2d[edge]
        best_fit = Legendre.fit(x, y, 5)
        coeffs = ",".join(map(str, best_fit.coef))
        domain = ",".join(map(str, best_fit.domain))
        range_str = ",".join(map(str, best_fit.window))
        print(f'Legendre(coef=[{coeffs}], domain=[{domain}], range=[{range_str}])') 

        orders['coj'][order_id]['coeffs'] = best_fit.coef
        orders['coj'][order_id]['domain'] = best_fit.domain
        orders['coj'][order_id]['window'] = best_fit.window
        order_id += 1
      

In [ ]:
test_files = ascii.read(os.path.join(importlib.resources.files('banzai_floyds.tests'), 'data/test_data.dat'))
for frame in test_files:
    frame_info = dict(frame)    
    hdu = fits.open(download_frame(frame_info['frameid'], frame_info['filename'], archive_api))
    hdu.writeto(os.path.join('manual_reduction', f'{frame_info["filename"]}'), overwrite=True)

In [ ]:
from glob import glob
from banzai_floyds.utils.order_utils import get_order_2d_region
from banzai_floyds.arc_lines import used_lines
from banzai_floyds.wavelengths import refine_peak_centers
import plotly.graph_objs as go
import plotly.express as px
import os 
import json 

order_height = 95

print([os.path.basename(filename) for filename in glob('manual_reduction/*a00.fits*')])

for filename in glob('manual_reduction/*a00.fits*'):
    hdu = fits.open(filename)
    for order in [1, 2]:
        x2d, y2d = np.meshgrid(np.arange(hdu['SCI'].data.shape[1]), np.arange(hdu['SCI'].data.shape[0]))
        site = hdu['SCI'].header['SITEID']
        order_center = Legendre(coef=orders[site][order]['coeffs'], domain=orders[site][order]['domain'],
                                window=orders[site][order]['window'])
        
        order_mask = np.logical_and(x2d >= min(order_center.domain), x2d <= max(order_center.domain))
        order_mask = np.logical_and(order_mask, y2d - order_center(x2d) >= -order_height // 2)
        order_mask = np.logical_and(order_mask, y2d  - order_center(x2d) <=  order_height // 2)
        order_region = get_order_2d_region(order_mask)
        
        starting_flux = hdu['SCI'].data[order_region][0]
        pixel = np.arange(len(starting_flux))
        fig = px.line(x=pixel, y=starting_flux)
        fig.show()

# Lines manually measured by Curtis 2024-09-09  (Same order as glob)

catalog_lines = {
    1: np.array([used_line['wavelength'] for used_line in used_lines[4:]]),
    2: np.array([used_line['wavelength'] for used_line in used_lines[:5]])
}


starting_line_locations = {
    'coj2m002-en12-20200813-0028-a00.fits.fz': 
        {1: np.array([295, 728, 758, 782, 816, 918, 1007, 1097, 1171, 
                1213, 1342, 1373, 1495, 1531, 1634]),
         2: np.array([183, 411, 431, 591, 1229])
         },
    'coj2m002-en12-20200813-0015-a00.fits.fz': 
        {1: np.array([297, 730, 759, 782, 818, 921, 1010, 1099, 1172,
                      1213, 1344, 1373, 1497, 1534, 1634]),
         2: np.array([183, 413, 430, 594, 1231]),
         },
    'ogg2m001-en06-20200822-0028-a00.fits.fz':
        {1: np.array([208, 644, 673, 695, 731, 836, 925, 1016, 1088, 1130,
                      1261, 1291, 1415, 1451, 1551]),
         2: np.array([201, 435, 456, 617, 1252]),
         },
    'ogg2m001-en06-20200822-0009-a00.fits.fz':
        {1: np.array([208,  644, 672, 696, 731, 836, 925, 1016, 1089, 1131,
             1262, 1291, 1414, 1450, 1551]),
         2: np.array([201, 435, 456, 618, 1252])
        }
    }


# Measured by eye by Curtis
line_fwhms = {
    'coj2m002-en12-20200813-0028-a00.fits.fz': {2: 596.2 - 589.5, 1: 1345.2 - 1340.2},
    'coj2m002-en12-20200813-0015-a00.fits.fz': {2: 597.7 - 590.8, 1: 1346.5 - 1341.5},
    'ogg2m001-en06-20200822-0028-a00.fits.fz': {2: 619.9 - 615.3, 1: 1263.4 - 1258.8},
    'ogg2m001-en06-20200822-0009-a00.fits.fz': {2: 620.25 - 615.4, 1: 1263.5 - 1258.75}
}
poly_order = {
    1: 5,
    2: 2
}

wavelength_fits = {
    'coj2m002-en12-20200813-0028-a00.fits.fz': {1: {'coef': [], 'domain': [], 'window': []}, 2: {'coef': [], 'domain': [], 'window': []}},
    'coj2m002-en12-20200813-0015-a00.fits.fz':  {1: {'coef': [], 'domain': [], 'window': []}, 2: {'coef': [], 'domain': [], 'window': []}},
    'ogg2m001-en06-20200822-0028-a00.fits.fz':  {1: {'coef': [], 'domain': [], 'window': []}, 2: {'coef': [], 'domain': [], 'window': []}},
    'ogg2m001-en06-20200822-0009-a00.fits.fz':  {1: {'coef': [], 'domain': [], 'window': []}, 2: {'coef': [], 'domain': [], 'window': []}}
}


# and then fit row by row, a set of lines measured by hand for all 4 arc lamps
for filename in starting_line_locations:
    hdu = fits.open(os.path.join('manual_reduction', filename))
    #then starting from the previous best fit, fit the next row
    x2d, y2d = np.meshgrid(np.arange(hdu['SCI'].data.shape[1]), np.arange(hdu['SCI'].data.shape[0]))
    site = hdu['SCI'].header['SITEID']
    for order in [1, 2]:
        figure = go.Figure()
        order_center = Legendre(coef=orders[site][order]['coeffs'], domain=orders[site][order]['domain'],
                                window=orders[site][order]['window'])
        
        order_mask = np.logical_and(x2d >= min(order_center.domain), x2d <= max(order_center.domain))
        order_mask = np.logical_and(order_mask, y2d - order_center(x2d) >= -order_height // 2)
        order_mask = np.logical_and(order_mask, y2d  - order_center(x2d) <=  order_height // 2)
        order_region = get_order_2d_region(order_mask)

        last_line_locations = starting_line_locations[filename][order]
        n_orig_lines = len(starting_line_locations[filename][order])

        # I have removed these lines in the current line list so no need to do it manually anymore
        # Remove the 3rd line in the blue because it is often blended and not high enough s/n
        #if order == 2:
        #    last_line_locations = last_line_locations[np.arange(n_orig_lines) != 2]
        #    this_catalog = catalog_lines[order][np.arange(n_orig_lines) != 2]
        # In the red, remove the 4th line which is too low of s/n and too close to nearby lines
        # I worry about the last line in the red, but it seems happier since it is isolated 
        #if order == 1:
        #    last_line_locations = last_line_locations[np.arange(n_orig_lines) != 3]
        #    this_catalog = catalog_lines[order][np.arange(n_orig_lines) != 3]

        for j in range(order_height):
            data = hdu['SCI'].data[order_region][j].astype(float)
            gain = float(hdu['SCI'].header['GAIN'])
            error = np.sqrt(hdu['SCI'].data[order_region][j].astype(float) * gain + hdu['SCI'].header['RDNOISE'] ** 2) / gain
            data -= np.median(data)
            measured_lines = refine_peak_centers(data, error, last_line_locations, line_fwhms[filename][order])
            if j == 0:
                pixel = np.arange(len(data))
                fig = px.line(x=pixel, y=data)
                for measured_line in measured_lines:
                    data_height = data[int(np.round(measured_line))]
                    fig.add_trace(go.Scatter(x=[measured_line, measured_line], y=[data_height * 1.1, data_height * 1.4], mode='lines', line=dict(color='salmon')))
                fig.show()
            best_fit = Legendre.fit(measured_lines, this_catalog, deg=poly_order[order],
                                    domain=(0, len(data) - 1))
            # save the Legendre polynomial for each row to include in the e2e test data
            wavelength_fits[filename][order]['coef'].append(list(best_fit.coef))
            wavelength_fits[filename][order]['domain'].append(list(orders[site][order]['domain']))
            wavelength_fits[filename][order]['window'].append(list(best_fit.window)) 
            last_line_locations = measured_lines
            # make a slider plot of each fit for each row to manually check each fit to make sure they look reasonable
            residuals = best_fit(measured_lines) - this_catalog
            figure.add_trace(go.Scatter(x=best_fit(measured_lines), y=residuals, mode='markers', name=f'{filename} Order {order} Row {j}', visible=(j == 0)))
        # Create slider steps
        steps = []
        for i in range(order_height):
            step = dict(
                method="update",
                args=[{"visible": [k == i for k in range(order_height)]},
                    {"title": f"Fit for Row {i}"}], 
            )
            steps.append(step)

        # Create slider
        sliders = [dict(
            active=0,
            currentvalue={"prefix": "Row: "},
            pad={"t": 50},
            steps=steps
        )]

        # Update layout with slider
        figure.update_layout(
            sliders=sliders,
            xaxis_title="Wavelength (Angstroms)",
            yaxis_title="Residuals (Angstroms)"
        )

        figure.show()
with open('../banzai_floyds/tests/data/wavelength_e2e_fits.dat', 'w') as f:
    f.writelines(json.dumps(wavelength_fits, indent=4))

TODO: check the 2d fft of the fringe image? I wonder if fitting phase parameters might actually work for the shifting to get rid of the lamp continuum

TODO: Define a regression test

TODO: Use specreduce to extract each source, with quadratic background

TODO: Regression test is that banzai extraction is within the noise of the manual extraction

TODO: Define regression tests for telluric correction and flux calibration